# 爱科研-人工智能科研实训
## 机器学习实战项目 - 猫狗大战

本项目会使用深度学习-卷积神经网络提取图片的特征向量，并借由机器学习模型来识别一张图片是猫还是狗，在验证集上的准确率可以达到99.3%。本项目采用的 Keras 版本为 2.1.0。

## 数据集

数据集来自 Kaggle 上的一个竞赛：[Dogs vs. Cats](https://www.kaggle.com/c/dogs-vs-cats-redux-kernels-edition)，训练集有25000张，猫狗各占一半。测试集12500张，没有标定是猫还是狗。

下面是训练集的一部分例子：

![](./images/dataset.png)

## 导出特征向量

对于这个项目来说，使用预训练的网络, 即迁移学习，是最好不过的了。经过前期的测试，我们测试了 ResNet50 等不同的网络，但是排名都不高，现在看来只有一两百名的样子，所以我们需要提高我们的模型表现。那么一种有效的方法是综合各个不同的模型，从而得到不错的效果，兼听则明。如果是直接在一个巨大的网络后面加我们的全连接，那么训练10代就需要跑十次巨大的网络，而且我们的卷积层都是不可训练的，那么这个计算就是浪费的。所以我们可以将多个不同的网络输出的特征向量先保存下来，以便后续的训练，这样做的好处是我们一旦保存了特征向量，即使是在普通笔记本上也能轻松训练。

In [ ]:
from keras.models import *
from keras.layers import *
from keras.applications import *
from keras.preprocessing.image import *

import h5py

In [ ]:
def write_gap(MODEL, image_size, lambda_func=None):
    width = image_size[0]
    height = image_size[1]
    input_tensor = Input((height, width, 3))
    x = input_tensor
    if lambda_func:
        x = Lambda(lambda_func)(x)

    base_model = MODEL(input_tensor=x, weights='imagenet', include_top=False)
    model = Model(base_model.input, GlobalAveragePooling2D()(base_model.output))

    gen = ImageDataGenerator()
    train_generator = gen.flow_from_directory("train", image_size, shuffle=False, 
                                              batch_size=16)
    test_generator = gen.flow_from_directory("test", image_size, shuffle=False, 
                                             batch_size=16, class_mode=None)

    train = model.predict_generator(train_generator, train_generator.samples)
    test = model.predict_generator(test_generator, test_generator.samples)
    with h5py.File("gap_%s.h5"%str(MODEL)[10:-23]) as h:
        h.create_dataset("train", data=train)
        h.create_dataset("test", data=test)
        h.create_dataset("label", data=train_generator.classes)

<b>说明</b>: 为了复用代码，写一个函数是非常有必要的，那么我们的函数就需要输入模型，输入图片的大小，以及预处理函数，因为 Xception 和 Inception V3 都需要将数据限定在 (-1, 1) 的范围内，然后我们利用 GlobalAveragePooling2D 将卷积层输出的每个激活图直接求平均值，不然输出的文件会非常大，且容易过拟合。然后我们定义了两个 generator，利用 model.predict_generator 函数来导出特征向量，最后我们选择了 ResNet50, Xception, Inception V3 这三个模型（如果有兴趣也可以导出 VGG 的特征向量）。每个模型导出的时间都挺长的，在 aws p2.xlarge 上大概需要用十分钟到二十分钟。 这三个模型都是在 ImageNet 上面预训练过的，所以每一个模型都可以说是身经百战，通过这三个老司机导出的特征向量，可以高度概括一张图片有哪些内容。

In [ ]:
write_gap(ResNet50, (224, 224)) # 不需要运行

In [ ]:
write_gap(InceptionV3, (299, 299), inception_v3.preprocess_input) # 不需要运行

In [ ]:
write_gap(Xception, (299, 299), xception.preprocess_input) # 不需要运行

最后导出的 h5 文件包括三个 numpy 数组：

- train (25000, 2048) -
- test (12500, 2048) - 
- label (25000,)-

如果你不想自己计算特征向量，可以直接在这里下载导出的文件：[百度云](https://pan.baidu.com/s/1OS-L3ABJhnts3lXqVZAt_Q)

参考资料：

* [ResNet](https://arxiv.org/abs/1512.03385) 15.12
* [Inception v3](https://arxiv.org/abs/1512.00567) 15.12
* [Xception](https://arxiv.org/abs/1610.02357) 16.10

## 载入特征向量

经过上面的代码以后，我们获得了三个特征向量文件，分别是：

* gap_ResNet50.h5
* gap_InceptionV3.h5
* gap_Xception.h5

我们需要选择性载入这些特征向量，然后记得把 X 和 y 打乱，不然之后我们设置`validation_split`的时候会出问题。这里设置了 numpy 的随机数种子为2017，这样可以确保每个人跑这个代码，输出都能是一样的结果。

In [ ]:
import numpy as np
from sklearn.utils import shuffle

features = []

for filename in ["gap_Xception.h5"]:
    with h5py.File(filename, 'r') as h:
        features.append(np.array(h['train']))
        labels = np.array(h['label'])

features = np.concatenate(features, axis=1)
features, labels = shuffle(features, labels)

In [ ]:
# 拆分训练集与验证集
from sklearn.cross_validation import train_test_split

#TODO: 使用train_test_split将数据集的80%划分为训练集，20%划分为验证集
X_train, X_val, y_train, y_val = train_test_split(None, None, test_size = None, random_state = 2)

print("Training set has {} samples.".format(len(X_train)))
print("Validation set has {} samples.".format(len(X_val)))

## 训练机器学习模型

#### 推荐模型

- [支持向量机](http://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html)
- [逻辑回归](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html)
- [梯度提升决策树](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html)

In [ ]:
#TODO: 从sklearn中调用机器学习模型并完成对训练集的训练
#进阶：使用网格搜索确认最佳超参数

model = None
model = model.fit(X_train, y_train)

In [ ]:
# 查看模型对验证集的精确度
from sklearn.metrics import recall_score, precision_score, accuracy_score

print("The accuracy score on validation set is {}".format(accuracy_score(y_val,model.predict(X_val))))

## 对测试集的猫狗图片进行预测

### 效果预览

![](./images/sample.png)

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
from PIL import Image
%matplotlib inline

from glob import glob
import random as rd

# 获取测试集图片路径
test_files = glob('./test/*')

In [ ]:
from keras.preprocessing import image 

# 将图像转化为张量
def image_to_tensor(img_path,image_size):
    img = image.load_img(img_path, target_size=image_size)
    x = image.img_to_array(img)
    return np.expand_dims(x, axis=0)

# 定义特征提取模型
def feature_exactor(image_size,MODEL,lambda_func=None):
    width = image_size[0]
    height = image_size[1]
    input_tensor = Input((height, width, 3))
    x = input_tensor
    if lambda_func:
        x = Lambda(lambda_func)(x)
    
    base_model = MODEL(input_tensor=x, weights='imagenet', include_top=False)
    model = Model(base_model.input, GlobalAveragePooling2D()(base_model.output))
    return model

In [ ]:
fig = plt.figure(figsize=(20, 11))
columns = 6
rows = 3
shape = (299, 299)
feature_model = feature_exactor(shape,Xception,xception.preprocess_input)

for i in range(1, columns*rows +1):
    img_id = rd.randint(0,len(test_files)-1)
    img = Image.open(test_files[img_id])
    img = img.resize(shape)
    fig.add_subplot(rows,columns,i)
    plt.imshow(img)
    img_feature = feature_model.predict(image_to_tensor(test_files[img_id],shape))
    img_label = model.predict(img_feature)
    title = 'dog' if img_label[0] == 1 else 'cat'
    plt.title('pred:%s' % (title))
plt.show()